In [1]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.append(str(project_root))
from visualization.utils import defaultColors
from helpers import load_from_path, save_image

In [2]:
# from mobo.surrogate_model import BoTorchSurrogateModelReapeat, BoTorchSurrogateModel
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import yaml

In [3]:
algos = ["qnehvi", "raqnehvi", "raqneirs"]
subfolder = "R10"
frmt = "pdf"
problem ="bstvert"
seed = 0

In [4]:
# Parameters
algos = ["qnehvi", "raqnehvi", "raqneirs"]
subfolder = "R10"
problem = "bstdiag"
seed = 0
frmt = "pdf"


In [5]:
path_dict = {}
for algo in algos:
    path_dict[algo] = Path(f"../result/{problem}/{subfolder}/{algo}/{seed}")

In [6]:
path = list(path_dict.values())[0]

res_dict = load_from_path(path)
eval_sample = res_dict["eval_samples"]
batch_size = res_dict["batch_size"]
init_samples = res_dict["init_samples"]
n_obj = res_dict["n_obj"]

index = eval_sample.index + 1

approx_all_dict = {} 
eval_samples_dict = {}

for algo, path in path_dict.items():
        
    approx_all_dict[algo] = load_from_path(path)["approx_all_df"]
    eval_samples_dict[algo] = load_from_path(path)["eval_samples"]


In [7]:

import sys
sys.path.append(r'..\problems')
sys.path.append('.')

from problems.common import get_problem_options
prob_class = get_problem_options()[path.parts[2]]
prob = prob_class(sigma=0.5, repeat_eval=100000, alpha=0.9)
alphas = [0.9]
Ys, Xs = prob.pareto_front(alphas=alphas, return_x=True)


SMOKE_TEST None


In [8]:
n_obj = prob.n_obj
bounds = prob.bounds
n = 30

x1 = np.linspace(bounds[0, 0], bounds[1, 0], n)
x2 = np.linspace(bounds[0, 1], bounds[1, 1], n)
X1, X2 = np.meshgrid(x1, x2)
X = np.stack([X1.flatten(), X2.flatten()]).T
Y = prob.f(X).reshape(n, n, 2)
rho_real = prob.get_noise_var(X).reshape(n, n, 2)
mvar = prob.evaluate_repeat_mvar(X).reshape(n, n, 2)

In [9]:
# %config InlineBackend.figure_format='svg'
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe

import matplotlib.gridspec as gridspec

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": ["Helvetica"],
        # font size
        "font.size": 16,
    }
)
labels_dict = {
    "qnehvi": "qNEHVI",
    "raqnehvi": "RAqNEHVI",
    "raqneirs": "RAqNEIRS",
}

colors = defaultColors

max_iter = eval_sample["iterID"].max()
iter = max_iter
z_lims = [[-0.15, 0.90], [-0.45, 1.2]]

labels = eval_sample[eval_sample["iterID"] < iter]["iterID"]
norm = plt.Normalize(labels.min(), labels.max())

objective_list = list(range(1, n_obj + 1))
key = "mvar_F_"
algos = list(approx_all_dict.keys())

Y_list_dict = {
    "F_1": Y[:, :, 0],
    "F_2": Y[:, :, 1],
    "rho_F_1": rho_real[:, :, 0],
    "rho_F_2": rho_real[:, :, 1],
    "mvar_F_1": mvar[:, :, 0],
    "mvar_F_2": mvar[:, :, 1],
}

x, y = X1, X2
size = 3
approx_all_i = Y_list_dict
rows = len(objective_list)
cols = len(algos)

gs = gridspec.GridSpec(
    rows + 1,
    cols + 1,
    width_ratios=[size] * cols + [0.2],
    height_ratios=[0.1] + [size] * rows,
)

fig = plt.figure(figsize=(size * (cols + 0.3), size * (rows + 0.3)))
subplot_axes = []

for j, algo in enumerate(algos):

    cmap = plt.cm.binary
    base_color = defaultColors[j]

    # colors = cmap(norm(labels.unique()))
    # all base color
    colors = [base_color] * len(labels)
    colors[0] = [0.7, 0.7, 0.7, 1]
    labels = labels.round(2)

    for i, obj_idx in enumerate(objective_list):

        current_key = f"{key}{obj_idx}"

        z = approx_all_i[current_key].reshape((n, n))

        # levels = np.linspace(z_lims[i][0], z_lims[i][1], 15)

        # round down and up to the nearest 0.1
        zmin_round = np.floor(z.min() * 10) / 10
        zmax_round = np.ceil(z.max() * 10) / 10

        levels = np.linspace(zmin_round, zmax_round, 20)


        ax = fig.add_subplot(
            gs[i + 1, j]
        )  # Start from the second row to avoid title overlap

        subplot_axes.append(ax)  # Add this axis to the list for later processing


        img = ax.contourf(x[0], y[:, 0], z, levels=levels, cmap="viridis")

        c = ax.contour(
            x[0],
            y[:, 0],
            z,
            levels=levels,
            colors="black",
            linestyles="solid",
            linewidths=0.5,
        )


        # ax.axvline(x=0.5, color="black", linestyle="--", linewidth=1)

        for _, X_paretos in enumerate(Xs):
            ax.plot(
                X_paretos[:, 0],
                X_paretos[:, 1],
                # label=f"$\\alpha={alphas[0]}$",
                color=defaultColors[10],
                marker="o",
                linestyle="",
                markersize=3,
                alpha=0.7,
            )

        for idx, label in enumerate(labels):

            eval_samples = eval_samples_dict[algo]

            x1 = eval_samples[eval_samples["iterID"] == label]["x1"][idx]

            x2 = eval_samples[eval_samples["iterID"] == label]["x2"][idx]


            mec, tc, marker = (
                ("white", "white", "o") if label == 0 else ("black", "black", "d")
            )

            name = "Sobol" if label == 0 else f"Optimization"

            ax.plot(

                x1,
                x2,
                color=colors[label],
                markeredgecolor=mec,
                linestyle="None",
                marker=marker,
                markersize=8,

                zorder=29,
                # label=name
            )


        if j == len(algos) - 1:
            cbar_ax = fig.add_subplot(gs[i + 1, -1])

            # make color bar have the same amount of ticks as the contour plot

            cbar = fig.colorbar(img, cax=cbar_ax, ticks=levels[::4])

            cbar.ax.tick_params(labelsize=10, direction="in", length=11, width=0.5)

            cbar.ax.yaxis.set_major_formatter(
                plt.FuncFormatter(lambda x, _: f"{x:.2f}")
            )

            cbar.ax.yaxis.grid(True, linestyle="--", linewidth=0.5)

            cbar.set_label(f"$VaR_{{0.9}}({{y_{obj_idx}}})$")

        if j == 0:

            ax.set_ylabel("$x_2$")

        if i == len(objective_list) - 1:

            ax.set_xlabel("$x_1$")



# Apply the label_outer and ticks settings only to the subplot axes



for ax in subplot_axes:

    ax.label_outer()

    ax.set(xticks=[x.min(), x.max()], yticks=[y.min(), y.max()])


for algo_idx, algo in enumerate(algos):
    ax.plot(
        [],
        [],
        color=defaultColors[algo_idx],
        linestyle="None",
        marker="d",
        markersize=8,
        label=labels_dict[algo],
    )

ax.plot(
    [],
    [],
    color=defaultColors[10],
    linestyle="None",
    marker="o",
    markersize=8,
    label="Optimal Pareto Set",
)
# sobol
# mec, tc, marker = (
#     ("white", "white", "o") if label == 0 else ("black", "black", "d")
# )


# name = "Sobol" if label == 0 else f"Optimization"

# ax.plot(

#     x1,
#     x2,
#     color=colors[label],
#     markeredgecolor=mec,
#     linestyle="None",
#     marker=marker,
#     markersize=8,
#     zorder=29,
#     # label=name
# )
# Sobol label
ax.plot(
    [],
    [],
    color=[0.7, 0.7, 0.7, 1],
    linestyle="None",
    marker="o",
    markersize=8,
    label="Sobol",
)


fig.legend(loc="lower center", ncol=len(algos), bbox_to_anchor=(0.5, -0.1))

save_image(
    fig,
    path,
    "IOplot_static_real_all",
    idx=f"{iter}+{'_'.join(algos)}+seed{seed}+descript",
    format=frmt,
    bbox_inches='tight'
)

WindowsPath('Plots_new/bstdiag/R10/IOplot_static_real_all_30+qnehvi_raqnehvi_raqneirs+seed0+descript.pdf')